In [1]:
import os
import cv2
import numpy as np
import random
import torch
import torch.optim as optim
import torchvision.datasets
import torchvision.transforms as T
from torchinfo import summary
# from torchsummary import summary 

In [2]:
MNIST_train = torchvision.datasets.MNIST(root='MNIST_data/', train=True, transform=T.ToTensor(), download=True)

In [3]:
MNIST_test = torchvision.datasets.MNIST(root='MNIST_data/', train=False, transform=T.ToTensor(), download=True)

In [4]:
MNIST_train.data.size()

torch.Size([60000, 28, 28])

In [5]:
MNIST_test.data.size()

torch.Size([10000, 28, 28])

In [ ]:
MNIST_train.data[0]

In [6]:
print(MNIST_train.targets[0])
sample_img = MNIST_train.data[0].numpy()
sample_img = cv2.resize(sample_img, (256, 256))
cv2.imshow('MNIST', sample_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

tensor(5)


img, label = next(iter(MNIST_train))

In [7]:
img, label = next(iter(MNIST_train))

In [8]:
img

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [9]:
label

5

In [10]:
batch_size = 100
training_epochs = 15
learning_rate = 0.001

In [11]:
data_loader = torch.utils.data.DataLoader(MNIST_train, batch_size=batch_size, shuffle=True)

In [12]:
data = next(iter(data_loader))
data[0] #  이미지배차서아주 먼쿰 거져옴
data[0][1] # 가져와서 0번째
data[1] # 레이블을 배치사이즈만큼 
data[1][0] # 가져온거 0번째

tensor(7)

In [13]:
total_batch = len(data_loader)
total_batch

600

In [14]:
a = torch.arange(10)
a.view(2,5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [15]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=128, out_channels=625, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc_layer = torch.nn.Linear(2 * 2 * 625, 10, bias=True)

    def forward(self, x):
        L1 = self.layer1(x)
        L2 = self.layer2(L1)
        L3 = self.layer3(L2)
        L4 = self.layer4(L3)
        fc_input = L4.view(L4.size(0), -1)
        fc = self.fc_layer(fc_input)
        return fc

In [16]:
model = CNN()

In [17]:
w1 = next(model.layer1.parameters())
w1.size()

torch.Size([32, 1, 3, 3])

summary(model, (1, 1, 28, 28))

In [18]:
summary(model, (1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      --                        --
├─Sequential: 1-1                        [1, 32, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─ReLU: 2-2                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 32, 14, 14]           --
├─Sequential: 1-2                        [1, 64, 7, 7]             --
│    └─Conv2d: 2-4                       [1, 64, 14, 14]           18,496
│    └─ReLU: 2-5                         [1, 64, 14, 14]           --
│    └─MaxPool2d: 2-6                    [1, 64, 7, 7]             --
├─Sequential: 1-3                        [1, 128, 3, 3]            --
│    └─Conv2d: 2-7                       [1, 128, 7, 7]            73,856
│    └─ReLU: 2-8                         [1, 128, 7, 7]            --
│    └─MaxPool2d: 2-9                    [1, 128, 3, 3]            --
├─Sequ

In [19]:
loss_func = torch.nn.CrossEntropyLoss()

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
for epoch in range(training_epochs):
    avg_cost = 0

    for i, data in enumerate(data_loader):
        X = data[0] # 학습이미지
        Y = data[1] # 학습 레이블
        optimizer.zero_grad() 
        hypothesis = model(X) 
        cost = loss_func(hypothesis, Y)
        cost.backward()
        optimizer.step()

        if i % 100 == 0:
            print('%d epochs, %d/%d batch' %(epoch + 1, i, total_batch))

        avg_cost += cost / total_batch

    print('[Epoch: %d, cost = %.9f' %(epoch + 1, avg_cost))

1 epochs, 0/600 batch
1 epochs, 100/600 batch
1 epochs, 200/600 batch
1 epochs, 300/600 batch
1 epochs, 400/600 batch
1 epochs, 500/600 batch
[Epoch: 1, cost = 0.190422252
2 epochs, 0/600 batch
2 epochs, 100/600 batch
2 epochs, 200/600 batch
2 epochs, 300/600 batch
2 epochs, 400/600 batch
2 epochs, 500/600 batch
[Epoch: 2, cost = 0.045665078
3 epochs, 0/600 batch
3 epochs, 100/600 batch
3 epochs, 200/600 batch
3 epochs, 300/600 batch
3 epochs, 400/600 batch
3 epochs, 500/600 batch
[Epoch: 3, cost = 0.030246703
4 epochs, 0/600 batch
4 epochs, 100/600 batch
4 epochs, 200/600 batch
4 epochs, 300/600 batch
4 epochs, 400/600 batch
4 epochs, 500/600 batch
[Epoch: 4, cost = 0.022936605
5 epochs, 0/600 batch
5 epochs, 100/600 batch
5 epochs, 200/600 batch
5 epochs, 300/600 batch
5 epochs, 400/600 batch
5 epochs, 500/600 batch
[Epoch: 5, cost = 0.018160494
6 epochs, 0/600 batch
6 epochs, 100/600 batch
6 epochs, 200/600 batch
6 epochs, 300/600 batch
6 epochs, 400/600 batch
6 epochs, 500/600 batc

softmax_func = torch.nn.Softmax(dim=1)

In [22]:
softmax_func = torch.nn.Softmax(dim=1)

In [23]:
transform = T.Compose([T.ToTensor()])

In [ ]:
r = torch.randint(0, len(MNIST_test), (1, 1)).item() # 1~ 10000사이의 임의의 값 전달

with torch.no_grad(): # 웨이트랑 바이오스 더 이상 변화시키지 않겠다
    X_test = MNIST_test.data[r].numpy()
    Y_test = MNIST_test.targets[r]

    test_img = cv2.resize(X_test, (256, 256))
    cv2.imshow('MNIST', test_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Label : %s' % Y_test.item())

    test_tensor_img = transform(X_test).unsqueeze(0) # 트랜스 폼하면 0,1 사이 값으로 변환, 언스키즈로 해줌(1장짜리 28바이 28로 함)
    prediction = model(test_tensor_img)
    print(prediction) # 스코어 반환 소프트맥스 전
    print(np.round(softmax_func(prediction).numpy(), 3))
    correct_prediction = torch.argmax(prediction, 1)
    print('prediction : %s' % correct_prediction.item())

In [ ]:
with torch.no_grad():
    for i in range(0, 10):
        img_path = './MNIST_TEST/image_' + str(i) +'.jpg'
        test_img = cv2.imread(img_path, 0)

        test_tensor_img = transform(test_img).unsqueeze(0)
        prediction = model(test_tensor_img)
        print(np.round(softmax_func(prediction).numpy(), 3))
        correct_prediction = torch.argmax(prediction, 1)
        print('prediction : %s \n' % correct_prediction.item())

        test_img = cv2.resize(test_img, (256, 256))
        cv2.imshow('MNIST', test_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [ ]:
random.seed(a=None)

In [ ]:
diagonal_img_RGB = np.zeros((280, 280, 3), np.uint8)
shuffled_imgs = np.arange(0, 10)

In [ ]:
cv2.imshow('MNIST', diagonal_img_RGB)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
random.shuffle(shuffled_imgs)
print(shuffled_imgs)
for i, img_idx in enumerate(shuffled_imgs):
    mnist_img = cv2.imread('MNIST_TEST/image_' + str(img_idx) + '.jpg')
    j = random.randint(0, 9)
    diagonal_img_RGB[i * 28:(i+1) * 28, j *28:(j+1) * 28] = mnist_img

In [ ]:
cv2.imshow('MNIST', diagonal_img_RGB)
cv2.waitKey(0)
cv2.destroyAllWindows()

diagonal_img_GRAY = cv2.cvtColor(diagonal_img_RGB, cv2.COLOR_BGR2GRAY)

In [ ]:
diagonal_img_GRAY = cv2.cvtColor(diagonal_img_RGB, cv2.COLOR_BGR2GRAY)

In [ ]:
with torch.no_grad():
    for i in range(0, 10):
        for j in range(0, 10):
            x1 = i * 28
            x2 = (i+1) * 28
            y1 = j * 28
            y2 = (j+1) * 28
            crop_img = diagonal_img_GRAY[x1:x2, y1:y2]

            test_tensor_img = transform(crop_img).unsqueeze(0)
            prediction = model(test_tensor_img)
            correct_prediction = torch.argmax(prediction, 1).item()
            prob = softmax_func(prediction)[0].numpy()

            if prob[correct_prediction] > 0.95:
                diagonal_img_RGB = cv2.rectangle(diagonal_img_RGB, (y1, x1), (y2, x2), (0, 255, 255), 2)
            else:
                diagonal_img_RGB = cv2.rectangle(diagonal_img_RGB, (y1, x1), (y2, x2), (0, 0, 255), 1)

        output_img = cv2.resize(diagonal_img_RGB, (560, 560))
        cv2.imshow('MNIST', output_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

model_save_path = './MNIST_model/MNIST_model.pth'

In [ ]:
model_save_path = './MNIST_model/MNIST_model.pth'

torch.save(model.state_dict(), model_save_path)

In [ ]:
torch.save(model.state_dict(), model_save_path)